In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

In [4]:
#ejercicio1

Leerlo sin intentar que Spark infiera el tipo de dato de cada columna
Puesto que existen columnas que contienen una coma enmedio del valor, en esos casos los valores vienen entre comillas dobles. Spark ya contempla esta posibilidad y puede leerlas adecuadamente si al leer le indicamos las siguientes opciones adicionales además de las que ya sueles usar: .option("quote", "\"").option("escape", "\"").
Asegúrate de que las filas que no tienen el formato correcto sean descartadas, indicando también la opción mode con el valor DROPMALFORMED como vimos en clase.
Crear un nuevo DF kivaRawNoNullDF en el que se hayan eliminado todas las filas que tengan algún valor nulo en cualquier columna excepto en la columna tags, que no será relevante para el análisis y por tanto podemos ignorar sus valores nulos y mantener dichas filas.

In [7]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
from pyspark.sql import functions as F
kivaRawDF = (spark.read.format("csv")\
        .option("header", "true")\
        .option("inferSchema", "false")\
        .option("quote", "\"").option("escape", "\"")\
        .option("mode", "DROPMALFORMED")\
        .load('C:/Users/laura.serrano/Desktop/kiva_loans.csv')).cache() 
kivaRawDF.printSchema()

# Descomentar estas líneas para calcular la lista de columnas que sí deben tenerse en cuenta para quitar nulos. Después
# tendrás que utilizar dicha lista en la operación que elimina los nulos
columnasExceptoTags = kivaRawDF.columns
columnasExceptoTags.remove("tags")
kivaRawNoNullDF = kivaRawDF.na.drop(subset=columnasExceptoTags)
# Elimina las filas que tenga nulos en todas las columnas excepto en tags 

root
 |-- id: string (nullable = true)
 |-- funded_amount: string (nullable = true)
 |-- loan_amount: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- use: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- partner_id: string (nullable = true)
 |-- posted_time: string (nullable = true)
 |-- disbursed_time: string (nullable = true)
 |-- funded_time: string (nullable = true)
 |-- term_in_months: string (nullable = true)
 |-- lender_count: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- borrower_genders: string (nullable = true)
 |-- repayment_interval: string (nullable = true)
 |-- date: string (nullable = true)



In [8]:
from pyspark.sql.types import DoubleType
assert(kivaRawNoNullDF.count() == 574115)

In [9]:
#EJERCICIO 2

Las columnas posted_time y disbursed_time son en realidad instantes de tiempo que Spark debería procesar como timestamp. Partiendo de kivaRawNoNullDF, reemplaza ambas columnas por su versión convertida a timestamp, utilizando withColumn con el mismo nombre de cada columna, y donde el nuevo valor de la columna viene dado por el siguiente código:

  F.from_unixtime(F.unix_timestamp('nombreColumna', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))
Además, convierte a DoubleType la columna loan_amount y a IntegerType la columna term_in_months.

El DF resultante de todas estas operaciones debe quedar almacenado en la variable kivaDF, cacheado.

In [18]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.types import DoubleType, IntegerType
# LÍNEAS EVALUABLES, NO RENOMBRAR LAS VARIABLES
kivaDF =(kivaRawNoNullDF.withColumn("posted_time",\
                                  F.from_unixtime(F.unix_timestamp('posted_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                       .withColumn("disbursed_time",\
                                  F.from_unixtime(F.unix_timestamp('disbursed_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                        .withColumn("loan_amount",F.col("loan_amount").cast(DoubleType()))\
                        .withColumn("term_in_months", F.col("term_in_months").cast(IntegerType())))\
.cache()

In [23]:
kivaDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- funded_amount: string (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- activity: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- use: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- partner_id: string (nullable = true)
 |-- posted_time: timestamp (nullable = true)
 |-- disbursed_time: timestamp (nullable = true)
 |-- funded_time: string (nullable = true)
 |-- term_in_months: integer (nullable = true)
 |-- lender_count: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- borrower_genders: string (nullable = true)
 |-- repayment_interval: string (nullable = true)
 |-- date: string (nullable = true)



In [19]:
typesDict = dict(kivaDF.dtypes)
assert(typesDict["posted_time"] == "timestamp") 
assert(typesDict["disbursed_time"] == "timestamp") 
assert(typesDict["loan_amount"] == "double") 
assert(typesDict["term_in_months"] == "int")
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
nullsRow = kivaDF.select(cnt_cond(F.col("posted_time").isNull()).alias("posted_nulls"),
              cnt_cond(F.col("disbursed_time").isNull()).alias("disbursed_nulls")).head()
assert(nullsRow.posted_nulls == 0)
assert(nullsRow.disbursed_nulls == 0)

In [10]:
#EJERCICIO 3

Partiendo de kivaDF:

Primero, añade una nueva columna dias_desembolso que contenga el número de días que han pasado entre la fecha en que los prestamistas aceptaron financiar un proyecto, y la fecha en que el agente de campo entregó los fondos al beneficiario. Para ello, utiliza withColumn en combinación con la función F.datediff("colFuturo", "colPasado")
De manera análoga, añade otra nueva columna dias_aceptacion que contenga el número de días entre el anuncio de la necesidad de préstamo y la aceptación de financiarlo por parte de algún prestamista.
Reemplazar la columna sector por otra en la que se hayan traducido las categorías "Education" por "Educacion" (sin tilde para evitar posibles problemas) y "Agriculture" por "Agricultura", dejando como están el resto de categorías. La sustitución no debe tener más que tres casos: uno para cada categoría que vamos a reemplazar, y un tercero para el resto de categorías, que deben quedarse como estaban.
El resultado debe quedar guardado en la variable kivaTiemposDF.

In [25]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# imports......
kivaTiemposDF = kivaDF.withColumn("dias_desembolso", F.datediff("disbursed_time", F.from_unixtime(F.unix_timestamp('funded_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp")))\
                      .withColumn("dias_aceptacion", F.datediff( F.from_unixtime(F.unix_timestamp('funded_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"), "posted_time"))\
                      .withColumn("Sector", F.when((F.col("Sector"))=="Education","Educacion").\
                                  when((F.col("sector") == "Agriculture"), "Agricultura")\
                                             .otherwise(F.col("sector")))

In [26]:
assert(kivaTiemposDF.where("sector == 'Agricultura'").count() == 157003)
assert(kivaTiemposDF.where("sector == 'Educacion'").count() == 28417)
# Comprobamos que las 13 restantes se mantienen sin cambios
assert(kivaTiemposDF.groupBy("sector").count().join(kivaDF.groupBy("sector").count(), ["sector", "count"]).count() == 13)

In [11]:
#EJERCICIO 4

Partiendo de kivaTiemposDF, crear un nuevo DataFrame llamado kivaAgrupadoDF que tenga:

Tantas filas como países (country; no usar el código de país), y tantas columnas como sectores (cada una llamada como el sector) más una (la columna del país, que debe aparecer en primer lugar). En cada celda deberá ir el número medio (redondeado a 2 cifras decimales) de días transcurridos en ese país y sector entre la fecha en que se anuncia la necesidad de préstamo y la fecha en la que un prestamista acepta financiarlo. Esta columna ha sido calculada en la celda precedente.
Después de esto, añadir una columna adicional transcurrido_global con el número medio (redondeado a 2 cifras decimales) de días transcurridos en cada país entre ambas fechas sin tener en cuenta el sector. Cada fila tendrá la media de las 15 columnas del apartado anterior.
Por último, ordenar el DF resultante descendentemente en base al tiempo medio global, transcurrido_global. El DF resultado de la ordenación debe ser almacenado en la variable kivaAgrupadoDF.
PISTA: utiliza el método pivot con el sector para el primer apartado, envolviendo a la función de agregación con la función F.round, es decir, F.round(F.funcionAgregacion(....), 2), y withColumn con una operación aritmética entre columnas en el segundo. No debe utilizarse la función when ya que Spark es capaz de hacer directamente aritmética entre objetos columna. La operación aritmética también debe estar envuelta por round: F.round(op. aritmética entre objetos columna, 2).

In [39]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
from functools import reduce
from operator import add
kivaAgrupadoDF1 = (kivaTiemposDF.groupBy("country").pivot("sector")\
                              .agg(F.round(F.avg(F.col("dias_aceptacion")), 2)))
kivaAgrupadoDF=kivaAgrupadoDF1.withColumn("transcurrido_global",F.round(reduce(add,\
                                    [F.col(x) for x in kivaAgrupadoDF1.columns[1:]])/(len(kivaAgrupadoDF1.columns) - 1), 2))\
.orderBy("transcurrido_global", ascending=False)

In [40]:
r1 = kivaAgrupadoDF.head()
assert(r1.country == "United States")
assert((r1.Agricultura - 12.0 < 0.01) | (r1.Agricultura - 12.17 < 0.01))
assert((r1.Educacion - 15.21 < 0.01) | (r1.Educacion - 15.33 < 0.01))
assert(r1.Wholesale - 27.5 < 0.01)
assert((r1.transcurrido_global - 20.94 < 0.01) | (r1.transcurrido_global - 21.04 < 0.01))

In [ ]:
#EJERCICIO 5

Partiendo de nuevo de kivaTiemposDF, añadir las siguientes columnas:

Primero, tres columnas adicionales llamadas transc_medio, transc_min, transc_max que contengan, respectivamente, el número de días medio, mínimo y máximo transcurrido para proyectos de ese mismo país y ese mismo sector entre la fecha en que se postea la necesidad de préstamo y la fecha en la que alguien acepta financiarlo (es decir, la columna dias_aceptacion calculada antes y utilizada también en la celda anterior). Es decir, queremos una columna extra para que podamos tener, junto a cada préstamo, información agregada de los préstamos similares, entendidos como aquellos del mismo país y del mismo sector. No se debe utilizar JOIN sino solo funciones de ventana.
Finalmente, crear otra columna adicional diff_dias que contenga la diferencia en días entre los días que transcurrieron en este proyecto y la media de días de los proyectos similares (calculada en el apartado anterior). Debería ser lo primero menos lo segundo, de forma que un número positivo indica que este préstamo tardó más días en ser aceptado que la media de días de este país y sector, y un número negativo indica lo contrario. El resultado debe obtenerse aplicando operaciones aritméticas con columnas existentes, sin utilizar when.
El DF resultante con las 4 columnas nuevas que hemos añadido debe quedar almacenado en la variable kivaExtraInfoDF.

In [41]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
from pyspark.sql import Window
windowPaisSector = Window().partitionBy("country", "sector")
kivaExtraInfoDF = kivaTiemposDF.withColumn("transc_medio", F.mean("dias_aceptacion").over(windowPaisSector))\
                               .withColumn("transc_min", F.min("dias_aceptacion").over(windowPaisSector))\
                               .withColumn("transc_max", F.max("dias_aceptacion").over(windowPaisSector))\
                               .withColumn("diff_dias",  ((F.datediff("funded_time", "posted_time")) - F.col("dias_aceptacion")))

In [42]:

r = kivaExtraInfoDF.where("id = '658540'").head()
assert(r.country == 'Burkina Faso')
assert(r.transc_medio - 11.02 < 0.05)
assert(r.dias_aceptacion == 35)
assert(r.diff_dias - 24.0 < 0.001)